In [26]:
import os
import requests
from datetime import datetime
import pandas as pd
import numpy as np
import json
from google.transit import gtfs_realtime_pb2


FUENTES = {
    "ACES": {
        "url": "https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-ace",
        "lineas": ["A", "C", "E", "Sr"]
    },
    "BDFMS": {
        "url": "https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-bdfm",
        "lineas": ["B", "D", "F", "M", "Sf"]
    },
    "G": {
        "url": "https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-g",
        "lineas": ["G"]
    },
    "JZ": {
        "url": "https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-jz",
        "lineas": ["J", "Z"]
    },
    "NQRW": {
        "url": "https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-nqrw",
        "lineas": ["N", "Q", "R", "W"]
    },
    "L": {
        "url": "https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-l",
        "lineas": ["L"]
    },
    "1234567S": {
        "url": "https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs",
        "lineas": ["1", "2", "3", "4", "5", "6", "7", "S"]
    },
    "SIR": {
        "url": "https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-si",
        "lineas": ["SIR"]
    }
}

In [2]:
def extraccion_linea(url, linea):
    """
    Extrae los datos de una línea
    """
    response = requests.get(url)
    fuentes = gtfs_realtime_pb2.FeedMessage()
    fuentes.ParseFromString(response.content)

    datos_linea = []
    for entity in fuentes.entity:
        if entity.HasField('trip_update'):
            trayecto = entity.trip_update

            if trayecto.trip.route_id == linea:
                for stop in trayecto.stop_time_update:
                    campos = {
                        'viaje_id': trayecto.trip.trip_id,
                        'linea_id': trayecto.trip.route_id,
                        'parada_id': stop.stop_id,
                        'hora_llegada': datetime.fromtimestamp(stop.arrival.time) if stop.HasField('arrival') else None,
                        'hora_partida': datetime.fromtimestamp(stop.departure.time) if stop.HasField('departure') else None,
                        'timestamp': datetime.now(),                       
                    }

                    datos_linea.append(campos)
    return datos_linea


In [3]:
def extraccion_datos():
    """
    Extrae todas las líneas y las convierte a DF
    """

    todas_las_lineas = []
    for info in FUENTES.values():
        todas_las_lineas.extend(info['lineas'])
    
    todos_los_datos = []
    for linea in todas_las_lineas:
        for grupo, info in FUENTES.items():
            if linea in info['lineas']:
                fuentes_url = info['url']
            todos_los_datos.extend(extraccion_linea(fuentes_url, linea))  

    return pd.DataFrame(todos_los_datos)
        

In [4]:
df = extraccion_datos()

In [5]:
df

,viaje_id,linea_id,parada_id,hora_llegada,hora_partida,timestamp
0,019900_A..S,A,H03S,2026-02-25 10:53:16,2026-02-25 10:53:16,2026-02-25 10:53:43.298969
1,019900_A..S,A,H17S,2026-02-25 10:55:34,2026-02-25 10:55:34,2026-02-25 10:53:43.298973
2,019900_A..S,A,H04S,2026-02-25 10:58:06,2026-02-25 10:58:06,2026-02-25 10:53:43.298975
3,019900_A..S,A,H18S,2026-02-25 11:00:28,2026-02-25 11:00:28,2026-02-25 10:53:43.298977
4,019900_A..S,A,H05S,2026-02-25 11:02:40,2026-02-25 11:02:40,2026-02-25 10:53:43.298979
...,...,...,...,...,...,...
53323,032550_7..N,7,707N,2026-02-25 11:52:40,2026-02-25 11:53:00,2026-02-25 10:54:56.289720
53324,032550_7..N,7,706N,2026-02-25 11:53:40,2026-02-25 11:54:00,2026-02-25 10:54:56.289721
53325,032550_7..N,7,705N,2026-02-25 11:54:40,2026-02-25 11:55:00,2026-02-25 10:54:56.289732
53326,032550_7..N,7,702N,2026-02-25 11:57:00,2026-02-25 11:58:30,2026-02-25 10:54:56.289734


In [6]:
df['hora_llegada'] = df['hora_llegada'].dt.tz_localize('UTC').dt.tz_convert('America/New_York')  
df['hora_partida'] = df['hora_partida'].dt.tz_localize('UTC').dt.tz_convert('America/New_York')
df['timestamp'] = df['timestamp'].dt.tz_localize('UTC').dt.tz_convert('America/New_York')

In [7]:
df['dia'] = df['timestamp'].dt.strftime("%A")
df['dia'] = df['dia'].apply(
    lambda x: 'Weekday' if x not in ('Saturday', 'Sunday') else x
)
df['hora_llegada'] = df['hora_llegada'].dt.strftime('%H:%M:%S')
df['hora_partida'] = df['hora_partida'].dt.strftime('%H:%M:%S')
df['timestamp'] = df['timestamp'].dt.strftime('%H:%M:%S')

In [8]:
df

,viaje_id,linea_id,parada_id,hora_llegada,hora_partida,timestamp,dia
0,019900_A..S,A,H03S,05:53:16,05:53:16,05:53:43,Weekday
1,019900_A..S,A,H17S,05:55:34,05:55:34,05:53:43,Weekday
2,019900_A..S,A,H04S,05:58:06,05:58:06,05:53:43,Weekday
3,019900_A..S,A,H18S,06:00:28,06:00:28,05:53:43,Weekday
4,019900_A..S,A,H05S,06:02:40,06:02:40,05:53:43,Weekday
...,...,...,...,...,...,...,...
53323,032550_7..N,7,707N,06:52:40,06:53:00,05:54:56,Weekday
53324,032550_7..N,7,706N,06:53:40,06:54:00,05:54:56,Weekday
53325,032550_7..N,7,705N,06:54:40,06:55:00,05:54:56,Weekday
53326,032550_7..N,7,702N,06:57:00,06:58:30,05:54:56,Weekday


In [9]:
norte = (df['parada_id'].str[-1] == 'N')
sur = (df['parada_id'].str[-1] == 'S')

df.loc[norte, 'direccion'] = 1
df.loc[sur, 'direccion'] = 0

df['direccion'] = df['direccion'].astype('Int64')


In [10]:
df

,viaje_id,linea_id,parada_id,hora_llegada,hora_partida,timestamp,dia,direccion
0,019900_A..S,A,H03S,05:53:16,05:53:16,05:53:43,Weekday,0
1,019900_A..S,A,H17S,05:55:34,05:55:34,05:53:43,Weekday,0
2,019900_A..S,A,H04S,05:58:06,05:58:06,05:53:43,Weekday,0
3,019900_A..S,A,H18S,06:00:28,06:00:28,05:53:43,Weekday,0
4,019900_A..S,A,H05S,06:02:40,06:02:40,05:53:43,Weekday,0
...,...,...,...,...,...,...,...,...
53323,032550_7..N,7,707N,06:52:40,06:53:00,05:54:56,Weekday,1
53324,032550_7..N,7,706N,06:53:40,06:54:00,05:54:56,Weekday,1
53325,032550_7..N,7,705N,06:54:40,06:55:00,05:54:56,Weekday,1
53326,032550_7..N,7,702N,06:57:00,06:58:30,05:54:56,Weekday,1


In [11]:
tot_filas = len(df)

for columna in df.columns:
    nulos = df[columna].isnull().sum()
    proporcion = nulos/tot_filas
    print(f"Proporcion nulos en {columna}: {proporcion}")

Proporcion nulos en viaje_id: 0.0
Proporcion nulos en linea_id: 0.0
Proporcion nulos en parada_id: 0.0
Proporcion nulos en hora_llegada: 0.006506900690069007
Proporcion nulos en hora_partida: 0.012976297629762976
Proporcion nulos en timestamp: 0.0
Proporcion nulos en dia: 0.0
Proporcion nulos en direccion: 0.0


In [12]:
df = df.dropna()
df

,viaje_id,linea_id,parada_id,hora_llegada,hora_partida,timestamp,dia,direccion
0,019900_A..S,A,H03S,05:53:16,05:53:16,05:53:43,Weekday,0
1,019900_A..S,A,H17S,05:55:34,05:55:34,05:53:43,Weekday,0
2,019900_A..S,A,H04S,05:58:06,05:58:06,05:53:43,Weekday,0
3,019900_A..S,A,H18S,06:00:28,06:00:28,05:53:43,Weekday,0
4,019900_A..S,A,H05S,06:02:40,06:02:40,05:53:43,Weekday,0
...,...,...,...,...,...,...,...,...
53322,032550_7..N,7,708N,06:51:40,06:52:00,05:54:56,Weekday,1
53323,032550_7..N,7,707N,06:52:40,06:53:00,05:54:56,Weekday,1
53324,032550_7..N,7,706N,06:53:40,06:54:00,05:54:56,Weekday,1
53325,032550_7..N,7,705N,06:54:40,06:55:00,05:54:56,Weekday,1


In [13]:
df_previsto = pd.read_csv('stop_times.txt', sep = ',')
df_previsto

,trip_id,stop_id,arrival_time,departure_time,stop_sequence
0,AFA25GEN-1038-Sunday-00_000600_1..S03R,101S,00:06:00,00:06:00,1
1,AFA25GEN-1038-Sunday-00_000600_1..S03R,103S,00:07:30,00:07:30,2
2,AFA25GEN-1038-Sunday-00_000600_1..S03R,104S,00:09:00,00:09:00,3
3,AFA25GEN-1038-Sunday-00_000600_1..S03R,106S,00:10:30,00:10:30,4
4,AFA25GEN-1038-Sunday-00_000600_1..S03R,107S,00:12:00,00:12:00,5
...,...,...,...,...,...
2484304,SIR-FA2017-SI017-Weekday-08_147100_SI..N03R,S27N,25:03:00,25:03:00,17
2484305,SIR-FA2017-SI017-Weekday-08_147100_SI..N03R,S28N,25:06:00,25:06:00,18
2484306,SIR-FA2017-SI017-Weekday-08_147100_SI..N03R,S29N,25:08:00,25:08:00,19
2484307,SIR-FA2017-SI017-Weekday-08_147100_SI..N03R,S30N,25:10:00,25:10:00,20


In [14]:
df_previsto['day'] = df_previsto['trip_id'].str.split('-').str[-2]
df_previsto['trip_id'] = df_previsto['trip_id'].str.split('_', n=1).str[-1]
df_previsto

,trip_id,stop_id,arrival_time,departure_time,stop_sequence,day
0,000600_1..S03R,101S,00:06:00,00:06:00,1,Sunday
1,000600_1..S03R,103S,00:07:30,00:07:30,2,Sunday
2,000600_1..S03R,104S,00:09:00,00:09:00,3,Sunday
3,000600_1..S03R,106S,00:10:30,00:10:30,4,Sunday
4,000600_1..S03R,107S,00:12:00,00:12:00,5,Sunday
...,...,...,...,...,...,...
2484304,147100_SI..N03R,S27N,25:03:00,25:03:00,17,Weekday
2484305,147100_SI..N03R,S28N,25:06:00,25:06:00,18,Weekday
2484306,147100_SI..N03R,S29N,25:08:00,25:08:00,19,Weekday
2484307,147100_SI..N03R,S30N,25:10:00,25:10:00,20,Weekday


In [15]:
def normalizar_horas(columna):
    columna = columna.str.replace('24:', '00:', regex=False)
    columna = columna.str.replace('25:', '01:', regex=False)
    columna = columna.str.replace('26:', '02:', regex=False)
    columna = columna.str.replace('27:', '03:', regex=False)
    return columna

df_previsto['arrival_time'] = normalizar_horas(df_previsto['arrival_time'])
df_previsto['departure_time'] = normalizar_horas(df_previsto['departure_time'])

In [16]:
def hora_a_segundos(hora):
    if pd.isna(hora): 
        return np.nan
    
    partes = hora.split(':')

    return int(partes[0]) * 3600 + int(partes[1]) * 60 + int(partes[2])

df['segundos_reales'] = df['hora_llegada'].apply(hora_a_segundos)
df_previsto['segundos_previstos'] = df_previsto['arrival_time'].apply(hora_a_segundos)

C:\Users\Sergio\AppData\Local\Temp\ipykernel_14692\3699567210.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['segundos_reales'] = df['hora_llegada'].apply(hora_a_segundos)


In [17]:
df_merge = pd.merge(df, df_previsto, left_on=['viaje_id', 'parada_id', 'dia'], right_on=['trip_id', 'stop_id', 'day'])

In [18]:
df_merge['delay'] = df_merge['segundos_reales']-df_merge['segundos_previstos']

In [19]:
def hora_posterior(hora1, hora2):
    partes1 = hora1.split(':')
    partes2 = hora2.split(':')

    return (
        (int(partes1[0]) > int(partes2[0])) | 
        ((int(partes1[0]) == int(partes2[0])) & (int(partes1[1]) > int(partes2[1]))) |
        ((int(partes1[0]) == int(partes2[0])) & (int(partes1[1]) == int(partes2[1])) & ((int(partes1[2]) > int(partes2[2]))))
    )

df_merge['delay'] = np.where(
    df_merge.apply(lambda row: hora_posterior(row['timestamp'], row['hora_llegada']), axis=1),
    df_merge['delay'],  # valor si True
    None    # valor si False
)

df_merge

,viaje_id,linea_id,parada_id,hora_llegada,hora_partida,timestamp,dia,direccion,segundos_reales,trip_id,stop_id,arrival_time,departure_time,stop_sequence,day,segundos_previstos,delay
0,018950_A..S74R,A,H09S,05:53:16,05:53:16,05:53:43,Weekday,0,21196,018950_A..S74R,H09S,05:03:00,05:03:00,56,Weekday,18180,3016
1,018950_A..S74R,A,H10S,05:54:31,05:54:31,05:53:43,Weekday,0,21271,018950_A..S74R,H10S,05:05:00,05:05:00,57,Weekday,18300,None
2,018950_A..S74R,A,H11S,05:56:01,05:56:01,05:53:43,Weekday,0,21361,018950_A..S74R,H11S,05:06:30,05:06:30,58,Weekday,18390,None
3,020200_A..N09R,A,A22N,05:53:16,05:53:16,05:53:43,Weekday,1,21196,020200_A..N09R,A22N,04:43:30,04:43:30,42,Weekday,17010,4186
4,020200_A..N09R,A,A21N,05:54:33,05:54:33,05:53:43,Weekday,1,21273,020200_A..N09R,A21N,04:45:00,04:45:00,43,Weekday,17100,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33790,032700_6..S01R,6,635S,07:17:30,07:17:30,05:54:52,Weekday,0,26250,032700_6..S01R,635S,06:17:30,06:17:30,33,Weekday,22650,None
33791,032700_6..S01R,6,636S,07:19:00,07:19:00,05:54:52,Weekday,0,26340,032700_6..S01R,636S,06:19:00,06:19:00,34,Weekday,22740,None
33792,032700_6..S01R,6,637S,07:20:30,07:20:30,05:54:52,Weekday,0,26430,032700_6..S01R,637S,06:20:30,06:20:30,35,Weekday,22830,None
33793,032700_6..S01R,6,638S,07:21:30,07:21:30,05:54:52,Weekday,0,26490,032700_6..S01R,638S,06:21:30,06:21:30,36,Weekday,22890,None


In [20]:
nulos = df_merge['delay'].isna().sum()
proporcion = nulos/(len(df_merge))
print(nulos, proporcion)

33154 0.981032697144548


In [21]:
df_merge.sort_values('hora_llegada')

,viaje_id,linea_id,parada_id,hora_llegada,hora_partida,timestamp,dia,direccion,segundos_reales,trip_id,stop_id,arrival_time,departure_time,stop_sequence,day,segundos_previstos,delay
27309,024950_4..S13R,4,402S,05:11:30,05:11:30,05:54:45,Weekday,0,18690,024950_4..S13R,402S,04:11:30,04:11:30,2,Weekday,15090,3600
27749,024950_4..S13R,4,402S,05:11:30,05:11:30,05:54:45,Weekday,0,18690,024950_4..S13R,402S,04:11:30,04:11:30,2,Weekday,15090,3600
29069,024950_4..S13R,4,402S,05:11:30,05:11:30,05:54:46,Weekday,0,18690,024950_4..S13R,402S,04:11:30,04:11:30,2,Weekday,15090,3600
29509,024950_4..S13R,4,402S,05:11:30,05:11:30,05:54:47,Weekday,0,18690,024950_4..S13R,402S,04:11:30,04:11:30,2,Weekday,15090,3600
28629,024950_4..S13R,4,402S,05:11:30,05:11:30,05:54:46,Weekday,0,18690,024950_4..S13R,402S,04:11:30,04:11:30,2,Weekday,15090,3600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20129,035100_R..S71R,R,R45S,08:15:00,08:15:00,05:54:27,Weekday,0,29700,035100_R..S71R,R45S,07:16:00,07:16:00,45,Weekday,26160,None
19722,035100_R..S71R,R,R45S,08:15:00,08:15:00,05:54:27,Weekday,0,29700,035100_R..S71R,R45S,07:16:00,07:16:00,45,Weekday,26160,None
19315,035100_R..S71R,R,R45S,08:15:00,08:15:00,05:54:27,Weekday,0,29700,035100_R..S71R,R45S,07:16:00,07:16:00,45,Weekday,26160,None
20536,035100_R..S71R,R,R45S,08:15:00,08:15:00,05:54:28,Weekday,0,29700,035100_R..S71R,R45S,07:16:00,07:16:00,45,Weekday,26160,None


In [22]:
df_merge['viaje_id'].nunique()

183

In [23]:
df_merge = df_merge.drop(['timestamp', 'segundos_reales', 'trip_id', 'stop_id', 'arrival_time', 'departure_time', 'day', 'segundos_previstos'], axis=1)
df_merge = df_merge.dropna()

In [24]:
df_merge

,viaje_id,linea_id,parada_id,hora_llegada,hora_partida,dia,direccion,stop_sequence,delay
0,018950_A..S74R,A,H09S,05:53:16,05:53:16,Weekday,0,56,3016
3,020200_A..N09R,A,A22N,05:53:16,05:53:16,Weekday,1,42,4186
54,022950_A..S74R,A,A49S,05:53:16,05:53:16,Weekday,0,39,3256
74,024200_A..N09R,A,A45N,05:53:11,05:53:11,Weekday,1,25,3821
153,024950_A..S74R,A,A33S,05:53:16,05:53:16,Weekday,0,26,3616
...,...,...,...,...,...,...,...,...,...
33039,025250_6..N01R,6,615N,05:54:46,05:54:46,Weekday,1,25,3556
33130,029250_6..N01R,6,637N,05:54:48,05:54:48,Weekday,1,4,3468
33417,023950_6..S01R,6,639S,05:54:27,05:59:57,Weekday,0,37,3567
33418,025250_6..N01R,6,615N,05:54:46,05:54:46,Weekday,1,25,3556


In [28]:
import io
from minio import Minio

# Leer credenciales de MinIO desde variables de entorno
ACCESS_KEY = os.getenv("MINIO_ACCESS_KEY")
assert ACCESS_KEY is not None, "La variable de entorno MINIO_ACCESS_KEY no está definida."
SECRET_KEY = os.getenv("MINIO_SECRET_KEY")
assert SECRET_KEY is not None, "La variable de entorno MINIO_SECRET_KEY no está definida."

# Crear un cliente de MinIO
client = Minio(
    endpoint="minio.fdi.ucm.es",
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
)

# Serializar el DataFrame a Parquet en memoria
buffer = io.BytesIO()
df_merge.to_parquet(buffer, index=False)
buffer.seek(0)

bucket = "pd1"
destination_file = "grupo5/processed/gtfs_real_time/mta_real_time.parquet"

client.put_object(
    bucket_name=bucket,
    object_name=destination_file,
    data=buffer,
    length=buffer.getbuffer().nbytes,
    content_type="application/octet-stream",
)

print(f"DataFrame subido a MinIO como '{destination_file}' en el bucket '{bucket}'.")

DataFrame subido a MinIO como 'grupo5/processed/gtfs_real_time/mta_real_time.parquet' en el bucket 'pd1'.
